# Word features in LLM exp 2

### preparation

In [ ]:
%pip install --upgrade openai tiktoken openpyxl --quiet

In [ ]:
# If you use Google Colab, you can run the following commands to clone the repository and navigate to the `llm` directory:

# !git clone https://github.com/hagi-hara/psychfeaturesllm.git
# %cd psychfeaturesllm/llm

In [ ]:
import os
# use your own API keys
# openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key = ""
# https://deepinfra.com/
# deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")
deepinfra_api_key = ""

In [ ]:
import pandas as pd
import json
import openai
from openai import AsyncOpenAI
from openai import OpenAI

import os
import time
import statistics

import nest_asyncio
import asyncio

nest_asyncio.apply()

In [ ]:
PROMPT_LIST_DF = pd.read_excel("./dataset/prompt_list.xlsx", header=0)

WORD_FEATURES_DF_EXP2 = pd.read_csv("./dataset/wordlist_exp2.csv", header=0)

WORD_FEATURES_DF_EXP2.arousal = WORD_FEATURES_DF_EXP2.arousal2
WORD_FEATURES_DF_EXP2.valence = WORD_FEATURES_DF_EXP2.valence2

In [ ]:
async def fetch_api(client, messages, model_id, model_name, feature_type, prop, i):
  '''Fetch API and save the response to a file.'''
  completion = await client.chat.completions.create(
    model= model_id,
    temperature=0.6,
    top_p=0.9,
    max_tokens=4096,
    # max_tokens=100000,
    messages=messages
  )

  with open(f"results_exp2/{model_name}/{model_name}_{feature_type}_{prop}_{i}.txt", mode="w") as f:
      f.write(completion.choices[0].message.content)

  with open(f"results_exp2/{model_name}/{model_name}_{feature_type}_{prop}_{i}.json", 'w') as f:
    json.dump(json.loads(completion.to_json()), f, indent=2)

  return completion


async def main(client, model_id, model_name, i,):
  '''Main function to fetch API for each word feature type and property ratio.'''
  fewshot_prompt = "For reference, the mean ratings by human participants are listed below. Please rate each word in accordance with the ratings provided."
  support_prompt = "Please answer about all of the following words."
  tasks = []
  for feature_type in PROMPT_LIST_DF.dropna().feature_type:
    # exclude socialness and imageability as they are missing
    if feature_type == "socialness" or feature_type == "imageability":
      continue
    prompt = PROMPT_LIST_DF[PROMPT_LIST_DF.feature_type==feature_type].iat[0, 1]
    for prop in ["10", "20", "30", "40", "50"]:
      prop_ratio_and_index = f'prop{prop}.{i}'
      training_index = WORD_FEATURES_DF_EXP2.loc[:, prop_ratio_and_index] == 'training'
      test_index = WORD_FEATURES_DF_EXP2.loc[:, prop_ratio_and_index] == 'test'
      test_words = WORD_FEATURES_DF_EXP2.loc[:, 'word_for_llm'][test_index].sample(frac=1, random_state=i)
      word_str_for_prompt = "\n".join(test_words.tolist())
      training_words_and_scores = WORD_FEATURES_DF_EXP2.loc[:, ['word_for_llm',feature_type]][training_index].sample(frac=1, random_state=i)
      # Round to the second decimal place from the third decimal place, and convert to a string.
      training_words_and_scores_str = '\n'.join([f"{row['word_for_llm']}: {round(row[feature_type], 2):.2f}" for _, row in training_words_and_scores.iterrows()])
      all_prompt = prompt + "\n\n" + fewshot_prompt + "\n\n" + training_words_and_scores_str +  "\n\n" + support_prompt + "\n\n" + word_str_for_prompt
      messages=[
        {"role": "system", "content": "You are a native English speaker."},
        {"role": "user", "content": all_prompt}
      ]
      with open(f"results_exp2/{model_name}/{model_name}_{feature_type}_{prop}_{i}_prompt.txt", mode="w") as f:
        f.write(all_prompt)
      tasks.append(fetch_api(client, messages, model_id, model_name, feature_type, prop, i))

  results = await asyncio.gather(*tasks)
  return results

In [ ]:
openapi_client = AsyncOpenAI(
    api_key=openai_api_key,
    base_url="https://api.openai.com/v1"
)
deepinfra_client = AsyncOpenAI(
    api_key=deepinfra_api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

model_list = [
     {
        "client": openapi_client,
        "model_id": "gpt-4o-2024-05-13",
        "model_name": "gpt-4o-2024-05-13"
     },
     {
        "client": deepinfra_client,
        "model_id": "meta-llama/Meta-Llama-3.1-70B-Instruct",
        "model_name": "meta-llama_Meta-Llama-3.1-70B-Instruct"
     },
]

### Run the main function

In [ ]:
num_of_iterations = 5
for model in model_list:
  for i in range(1, 1 + num_of_iterations):
    client = model["client"]
    model_id = model["model_id"]
    model_name = model["model_name"]

    print(f"model_name: {model_name}, iteration: {i}")
    os.makedirs(f"results_exp2/{model_name}/", exist_ok=True)
    results = asyncio.run(main(client, model_id, model_name, i))

### Summarize the results of the output

In [ ]:
def get_feature_results(model_name, num_of_iterations):
  '''get feature results from the saved files by parsing the output files.'''

  word_score_df_list = []

  for feature_type in PROMPT_LIST_DF.dropna().feature_type:
    if feature_type == "socialness" or feature_type == "imageability":
      continue
    prompt = PROMPT_LIST_DF[PROMPT_LIST_DF.feature_type==feature_type].iat[0, 1]
    for prop in ["50", "40", "30", "20", "10"]:
      for i in range(1, 1 + num_of_iterations):

        # load file
        file_name = f"results_exp2/{model_name}/{model_name}_{feature_type}_{prop}_{i}.json"
        with open(file_name, 'r') as f:
          llm_output = json.load(f)['choices'][0]['message']['content']

          # parse output
          output_dict = {}
          for output in  llm_output.splitlines():
            try:
              word, score = output.split(': ')
            except Exception as e:
              word = output
              score = output
            try:
              # output_dict[word] = int(score)
              output_dict[word] = float(score)
            except:
              output_dict[word] = score

          # Order according to word_for_llm. For words not outputted, set the score to X. For training words, set it to T.
          reordered_dict = {}
          prop_ratio_and_index = f'prop{prop}.{i}'
          training_index = WORD_FEATURES_DF_EXP2.loc[:, prop_ratio_and_index] == 'training'
          training_words = WORD_FEATURES_DF_EXP2.loc[:, 'word_for_llm'][training_index]
          for word in WORD_FEATURES_DF_EXP2.loc[:, 'word_for_llm'].unique():
            if word in training_words.tolist():
              reordered_dict[word] = "T"
              try:
                print(output_dict[word])
                print("This is a training word")
              except:
                pass
              continue
            try:
              reordered_dict[word] = output_dict[word]
            except:
              reordered_dict[word] = "X"

          word_score_df = pd.Series(reordered_dict, name=f"{model_name}_{feature_type}_{prop}_{i}")
          word_score_df_list.append(word_score_df)

  all_results_df = pd.concat(word_score_df_list, axis=1)

  # Calculate the average of each word feature
  for feature_type in PROMPT_LIST_DF.dropna().feature_type:
    if feature_type == "socialness" or feature_type == "imageability":
      continue
    for prop in ["50", "40", "30", "20", "10"]:
      tmp_df = all_results_df[[f"{model_name}_{feature_type}_{prop}_{i}" for i in range(1, 1 + num_of_iterations)]]
      tmp_df.replace('N', pd.NA, inplace=True)
      tmp_df.replace('X', pd.NA, inplace=True)
      tmp_df.replace('T', pd.NA, inplace=True)
      average = tmp_df.apply(pd.to_numeric, errors='coerce').mean(axis=1)
      average.name = f"{model_name}_{feature_type}_{prop}_average"
      all_results_df.loc[:, average.name] = average

  all_results_df.index.name = 'word_used'
  all_results_df = all_results_df.reset_index()

  return all_results_df

### Save the results to csv file

In [ ]:
model_name_list = [model["model_name"] for model in model_list]

num_of_iterations = 5
all_results_df_list = []
for model_name in model_name_list:
  all_results_df = get_feature_results(model_name, num_of_iterations)
  all_results_df_list.append(all_results_df)
  print(model_name)
  
concatenated_results_df = pd.concat([WORD_FEATURES_DF_EXP2] + all_results_df_list, axis=1)

concatenated_results_df.to_csv('results_exp2/concatenated_results.csv')